In [28]:
import glob

from src.create_annotations import *

# Label ids of the dataset
category_ids = {
    "rebar": 0,
    "spall": 1,
    "crack": 2,
}

# Define which colors match which categories in the images
category_colors = {
    "(0, 0, 0)": 0, # Outlier
    "(255, 0, 0)": 1, # Window
    "(255, 255, 0)": 2, # Wall
}

# Define the ids that are a multiplolygon. In our case: wall, roof and sky
# multipolygon_ids = [ 0, 1, 2]
multipolygon_ids = [5]

# Get "images" and "annotations" info 
def images_annotations_info(maskpath):
    # This id will be automatically increased as we go
    annotation_id = 0
    image_id = 0
    annotations = []
    images = []

    for mask_image in glob.glob(maskpath + "*.jpg"):
        # The mask image is *.png but the original image is *.jpg.
        # We make a reference to the original file in the COCO JSON file
        original_file_name = os.path.basename(mask_image).split(".")[0] + ".jpg"

        # Open the image and (to be sure) we convert it to RGB
        mask_image_open = Image.open(mask_image).convert("RGB")
        w, h = mask_image_open.size
        
        # "images" info
        image = create_image_annotation(original_file_name, w, h, image_id)
        images.append(image)


        sub_masks = create_sub_masks(mask_image_open, w, h)
        for color, sub_mask in sub_masks.items():
            if "rebar" in mask_image:
                category_id = 0
            if "spall" in mask_image:
                category_id = 0
            if "crack" in mask_image:
                category_id = 0
            # category_id = category_colors[color]

            # "annotations" info
            polygons, segmentations = create_sub_mask_annotation(sub_mask)
            # print(segmentations)
            # Check if we have classes that are a multipolygon
            if category_id in multipolygon_ids:
                # Combine the polygons to calculate the bounding box and area
                multi_poly = MultiPolygon(polygons)
                print("multi_poly.bounds",multi_poly.bounds)
                annotation = create_annotation_format(multi_poly, segmentations, image_id, category_id, annotation_id)

                annotations.append(annotation)
                annotation_id += 1
            else:
                print(len(polygons))
                for i in range(len(polygons)):
                    # Cleaner to recalculate this variable
                    segmentation = [np.array(polygons[i].exterior.coords).ravel().tolist()]
                    
                    annotation = create_annotation_format(polygons[i], segmentation, image_id, category_id, annotation_id)
                    
                    annotations.append(annotation)
                    annotation_id += 1
        image_id += 1
    return images, annotations, annotation_id


In [29]:
if __name__ == "__main__":
    # Get the standard COCO JSON format
    coco_format = get_coco_json_format()
    root = "/home/francis/Documents/Atom360/code/Dataset/buildings/Building dataset/"
    
    for keyword in ["train"]:
        mask_path = root+"dataset/masks/"
        
        # Create category section
        coco_format["categories"] = create_category_annotation(category_ids)
    
        # Create images and annotations sections
        coco_format["images"], coco_format["annotations"], annotation_cnt = images_annotations_info(mask_path)

        with open("output/{}.json".format(keyword),"w") as outfile:
            json.dump(coco_format, outfile)
        
        print("Created %d annotations for images in folder: %s" % (annotation_cnt, mask_path))

AttributeError: 'MultiPolygon' object has no attribute 'exterior'